In [1]:
!python -m spacy download es_core_news_sm

import re

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize,wordpunct_tokenize,sent_tokenize
import spacy 
import es_core_news_sm

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 7.2 MB/s  0:00:01a 0:00:01m eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


[nltk_data] Downloading package stopwords to /home/aldair/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aldair/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/aldair/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Exploración

In [2]:
df = pd.read_csv("noticiasUAM-10.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   titulo         10 non-null     object
 1   resumen        10 non-null     object
 2   nivel_impacto  10 non-null     object
dtypes: object(3)
memory usage: 372.0+ bytes
None


In [3]:
textos = df.resumen
textos.head()

0     Ciudad de México; 20 de mayo.- Esta mañana se...
1     Luego de confirmar el hecho durante su confer...
2     La ola de calor que se registra actualmente e...
3     La Universidad Autónoma Metropolitana ( UAM )...
4     A cien años del nacimiento de aquel "muchacho...
Name: resumen, dtype: object

In [4]:
textos[3]

' La Universidad Autónoma Metropolitana ( UAM ) ya compartió su convocatoria para el Pase UAM 2025 el cual los estudiantes del Colegio de Bachilleres ( Colbach ) podrán tener un mecanismo de acceso directo siempre y cuando cumplan con los requisitos necesarios como el promedio'

## Limpieza
- Eliminar menciones de usuarios (@user)
- Eliminar caracteres especiales +/?!¡;
- Eliminar correos electrónicos
- Eliminar direcciones URL

In [5]:
usuarios = r"(?<!\w)@\w+"
email = r"\w+@\w+[.\w]+(?=\s)"
especiales = r"[+/?!¡;\(\)\.\-]"
url = r"https?://.+(?=\s)"
todos = f"({url})|({email})|({usuarios})|({especiales})"

expr = re.compile(todos, re.M | re.I)
textos_limpios = textos.str.replace(
    pat=expr,
    repl="",
    regex=True
)

In [6]:
print(textos[3])
print("*" * 100)
print(textos_limpios[3])

 La Universidad Autónoma Metropolitana ( UAM ) ya compartió su convocatoria para el Pase UAM 2025 el cual los estudiantes del Colegio de Bachilleres ( Colbach ) podrán tener un mecanismo de acceso directo siempre y cuando cumplan con los requisitos necesarios como el promedio
****************************************************************************************************
 La Universidad Autónoma Metropolitana  UAM  ya compartió su convocatoria para el Pase UAM 2025 el cual los estudiantes del Colegio de Bachilleres  Colbach  podrán tener un mecanismo de acceso directo siempre y cuando cumplan con los requisitos necesarios como el promedio


Note la ausencia de paréntesis entre el primer texto y el segundo.

## Normalizar
- Pasar todo a minúsculas

In [7]:
textos_limpios = textos_limpios.str.lower()
textos_limpios

0     ciudad de méxico 20 de mayo esta mañana se in...
1     luego de confirmar el hecho durante su confer...
2     la ola de calor que se registra actualmente e...
3     la universidad autónoma metropolitana  uam  y...
4     a cien años del nacimiento de aquel "muchacho...
5     ciudad de méxico investigadores de la univers...
6     el instituto nacional electoral ine ha suscri...
7     académicos magistrados y expertos del sector ...
8     cinco facultades de la universidad autónoma m...
9     prepárate porque este miércoles 7 de mayo de ...
Name: resumen, dtype: object

## Palabras vacías (Stopwords)

In [8]:
from nltk.corpus import stopwords

palabras_vacias = set(stopwords.words("spanish"))

In [9]:
def quitar_palabras_vacias(texto: str):
    tokens = texto.split()
    filtrado = [token for token in tokens if token not in palabras_vacias]
    return " ".join(filtrado)

In [10]:
textos_limpios = textos_limpios.apply(quitar_palabras_vacias)
textos_limpios

0    ciudad méxico 20 mayo mañana informó asesinato...
1    luego confirmar hecho conferencia prensa matut...
2    ola calor registra actualmente ciudad méxico o...
3    universidad autónoma metropolitana uam compart...
4    cien años nacimiento aquel "muchacho genial fl...
5    ciudad méxico investigadores universidad nacio...
6    instituto nacional electoral ine suscrito conv...
7    académicos magistrados expertos sector financi...
8    cinco facultades universidad autónoma metropol...
9    prepárate miércoles 7 mayo 2025 prevén menos t...
Name: resumen, dtype: object

In [11]:
print(textos[3])
print("*" * 100)
print(textos_limpios[3])

 La Universidad Autónoma Metropolitana ( UAM ) ya compartió su convocatoria para el Pase UAM 2025 el cual los estudiantes del Colegio de Bachilleres ( Colbach ) podrán tener un mecanismo de acceso directo siempre y cuando cumplan con los requisitos necesarios como el promedio
****************************************************************************************************
universidad autónoma metropolitana uam compartió convocatoria pase uam 2025 estudiantes colegio bachilleres colbach podrán tener mecanismo acceso directo siempre cumplan requisitos necesarios promedio


Note la ausencia de artículos y de preposiciones.

## Segmentar 
- Unigramas

In [12]:
unigramas = textos_limpios.apply(lambda texto: word_tokenize(texto, language="spanish")) 
unigramas[3]

['universidad',
 'autónoma',
 'metropolitana',
 'uam',
 'compartió',
 'convocatoria',
 'pase',
 'uam',
 '2025',
 'estudiantes',
 'colegio',
 'bachilleres',
 'colbach',
 'podrán',
 'tener',
 'mecanismo',
 'acceso',
 'directo',
 'siempre',
 'cumplan',
 'requisitos',
 'necesarios',
 'promedio']

In [13]:
procesado = pd.DataFrame({
    "texto_original": textos,
    "unigramas": unigramas
})
procesado

,texto_original,unigramas
0,Ciudad de México; 20 de mayo.- Esta mañana se...,"[ciudad, méxico, 20, mayo, mañana, informó, as..."
1,Luego de confirmar el hecho durante su confer...,"[luego, confirmar, hecho, conferencia, prensa,..."
2,La ola de calor que se registra actualmente e...,"[ola, calor, registra, actualmente, ciudad, mé..."
3,La Universidad Autónoma Metropolitana ( UAM )...,"[universidad, autónoma, metropolitana, uam, co..."
4,"A cien años del nacimiento de aquel ""muchacho...","[cien, años, nacimiento, aquel, ``, muchacho, ..."
5,CIUDAD DE MÉXICO. Investigadores de la Univer...,"[ciudad, méxico, investigadores, universidad, ..."
6,El Instituto Nacional Electoral (INE) ha susc...,"[instituto, nacional, electoral, ine, suscrito..."
7,Académicos magistrados y expertos del sector ...,"[académicos, magistrados, expertos, sector, fi..."
8,Cinco facultades de la Universidad Autónoma M...,"[cinco, facultades, universidad, autónoma, met..."
9,¡Prepárate! Porque este miércoles 7 de mayo d...,"[prepárate, miércoles, 7, mayo, 2025, prevén, ..."


## Lematización

In [14]:
nlp = es_core_news_sm.load()

def crear_lemas(texto: str):
    return [palabra.lemma_ for palabra in nlp(texto)]

lemas = textos_limpios.apply(crear_lemas)
lemas[3]

['universidad',
 'autónoma',
 'metropolitano',
 'uam',
 'compartir',
 'convocatorio',
 'pasar',
 'uam',
 '2025',
 'estudiante',
 'colegio',
 'bachiller',
 'colbach',
 'poder',
 'tener',
 'mecanismo',
 'acceso',
 'directo',
 'siempre',
 'cumplir',
 'requisito',
 'necesario',
 'promedio']

## Derivación (Stemming)

In [15]:
from nltk import SnowballStemmer

esp = SnowballStemmer("spanish")

def crear_stemming(tokens: list):
    tokens = list(tokens)
    return [esp.stem(token) for token in tokens]    

stemming = unigramas.apply(crear_stemming)
stemming[3]

['univers',
 'autonom',
 'metropolitan',
 'uam',
 'compart',
 'convocatori',
 'pas',
 'uam',
 '2025',
 'estudi',
 'colegi',
 'bachiller',
 'colbach',
 'podran',
 'ten',
 'mecan',
 'acces',
 'direct',
 'siempr',
 'cumpl',
 'requisit',
 'necesari',
 'promedi']

## Resultados

In [16]:
procesado = pd.DataFrame({
    "texto_original": textos,
    "texto_limpio": textos_limpios,
    "unigramas": unigramas,
    "lemas": lemas,
    "stemming": stemming,
})
procesado

,texto_original,texto_limpio,unigramas,lemas,stemming
0,Ciudad de México; 20 de mayo.- Esta mañana se...,ciudad méxico 20 mayo mañana informó asesinato...,"[ciudad, méxico, 20, mayo, mañana, informó, as...","[ciudad, méxico, 20, mayo, mañana, informar, a...","[ciud, mexic, 20, may, mañan, inform, asesinat..."
1,Luego de confirmar el hecho durante su confer...,luego confirmar hecho conferencia prensa matut...,"[luego, confirmar, hecho, conferencia, prensa,...","[luego, confirmar, hecho, conferencia, prensa,...","[lueg, confirm, hech, conferent, prens, matuti..."
2,La ola de calor que se registra actualmente e...,ola calor registra actualmente ciudad méxico o...,"[ola, calor, registra, actualmente, ciudad, mé...","[ola, calor, registrar, actualmente, ciudad, m...","[ola, calor, registr, actual, ciud, mexic, obe..."
3,La Universidad Autónoma Metropolitana ( UAM )...,universidad autónoma metropolitana uam compart...,"[universidad, autónoma, metropolitana, uam, co...","[universidad, autónoma, metropolitano, uam, co...","[univers, autonom, metropolitan, uam, compart,..."
4,"A cien años del nacimiento de aquel ""muchacho...","cien años nacimiento aquel ""muchacho genial fl...","[cien, años, nacimiento, aquel, ``, muchacho, ...","[cien, año, nacimiento, aquel, "", muchacho, ge...","[cien, años, nacimient, aquel, ``, muchach, ge..."
5,CIUDAD DE MÉXICO. Investigadores de la Univer...,ciudad méxico investigadores universidad nacio...,"[ciudad, méxico, investigadores, universidad, ...","[ciudad, méxico, investigadores, universidad, ...","[ciud, mexic, investig, univers, nacional, aut..."
6,El Instituto Nacional Electoral (INE) ha susc...,instituto nacional electoral ine suscrito conv...,"[instituto, nacional, electoral, ine, suscrito...","[instituto, nacional, electoral, ine, suscrito...","[institut, nacional, electoral, ine, suscrit, ..."
7,Académicos magistrados y expertos del sector ...,académicos magistrados expertos sector financi...,"[académicos, magistrados, expertos, sector, fi...","[académico, magistrado, experto, sector, finan...","[academ, magistr, expert, sector, financier, b..."
8,Cinco facultades de la Universidad Autónoma M...,cinco facultades universidad autónoma metropol...,"[cinco, facultades, universidad, autónoma, met...","[cinco, facultad, universidad, autónoma, metro...","[cinc, facultad, univers, autonom, metropolita..."
9,¡Prepárate! Porque este miércoles 7 de mayo d...,prepárate miércoles 7 mayo 2025 prevén menos t...,"[prepárate, miércoles, 7, mayo, 2025, prevén, ...","[prepáratir, miércoles, 7, mayo, 2025, prever,...","[preparat, miercol, 7, may, 2025, preven, men,..."
